# Drone Video: Gaussian Splatting
powerd by gemini

In [ ]:
!pip install imageio-ffmpeg
!pip install pycolmap

In [ ]:
import os, sys
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pycolmap
from pathlib import Path
from PIL import Image
from ipywidgets import interactive, widgets

In [ ]:
path = '/kaggle/input/drone-video-pycolmap-camera-position'
recon = pycolmap.Reconstruction(f'{path}/sparse/0')
image_dir = Path(f'{path}/images/')

In [ ]:
all_images = []
all_poses = []
all_intrinsics = []

for image_id, image in recon.images.items():
    # Load image
    img_path = image_dir / image.name
    img_pil = Image.open(img_path)
    img = np.array(img_pil) / 255.0
    all_images.append(img)

    # Intrinsic matrix
    cam = recon.cameras[image.camera_id]
    fx, fy = cam.params[0], cam.params[1]
    cx, cy = cam.params[2], cam.params[3]
    intrinsics = np.array([
        [fx, 0, cx],
        [0, fy, cy],
        [0,  0,  1]
    ])
    all_intrinsics.append(intrinsics)

    # Get camera pose (world-to-camera transformation)
    cam_fromworld = image.cam_from_world()

    # Extract rotation and translation
    R = cam_fromworld.rotation.matrix()  # Rotation matrix (3x3)
    t = cam_fromworld.translation        # Translation vector (3,)

    # Convert to camera-to-world (invert the transformation)
    c2w = np.eye(4)
    c2w[:3, :3] = R.T              # Transpose of rotation
    c2w[:3, 3] = -R.T @ t          # New translation
    all_poses.append(c2w)

images = np.stack(all_images)
poses = np.stack(all_poses)
intrinsics = np.stack(all_intrinsics)

print('images.shape:', images.shape)
print('poses.shape:', poses.shape)

H, W = images.shape[1:3]
print('Image dimensions (H, W):', H, W)

# Extract camera positions
camera_positions = poses[:, :3, 3]
print("First 3 camera positions:")
print(camera_positions[0:3])

camera_rotations = poses[:, :3, :3]  # Shape: (N, 3, 3)
camera_directions = -camera_rotations[:, :, 2]  # Shape: (N, 3)
radius = np.linalg.norm(camera_positions, axis=1)
print("Camera distances from origin:")
print(radius)

In [ ]:
# Get any image
sample_image = next(iter(recon.images.values()))
camera = recon.cameras[sample_image.camera_id]

# Get camera model information correctly
camera_model = camera.model  # This returns the model type directly
print('Camera model:', camera_model)
print('Camera parameters:', camera.params)

# Extract focal length depending on model
if camera_model == 'SIMPLE_PINHOLE':
    focal = camera.params[0]       # [f, cx, cy]
elif camera_model == 'PINHOLE':
    fx = camera.params[0]          # [fx, fy, cx, cy]
    fy = camera.params[1]
    focal = (fx + fy) / 2
elif camera_model == 'SIMPLE_RADIAL':
    focal = camera.params[0]       # [f, cx, cy, k]
elif camera_model == 'RADIAL':
    focal = camera.params[0]       # [f, cx, cy, k1, k2]
elif camera_model == 'OPENCV':
    fx = camera.params[0]          # [fx, fy, cx, cy, k1, k2, p1, p2]
    fy = camera.params[1]
    focal = (fx + fy) / 2
elif camera_model == 'OPENCV_FISHEYE':
    fx = camera.params[0]          # [fx, fy, cx, cy, k1, k2, k3, k4]
    fy = camera.params[1]
    focal = (fx + fy) / 2
else:
    # For unknown models, assume first parameter is focal length
    print(f"Warning: Unknown camera model {camera_model}, assuming first parameter is focal length")
    focal = camera.params[0]

print('Focal length:', focal)

---

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(
    camera_positions[:, 0],
    camera_positions[:, 1],
    camera_positions[:, 2],
    c='blue',
    label='Camera Position'
)

for i in range(len(poses)):
    ax.quiver(
        camera_positions[i, 0],  # point(x)
        camera_positions[i, 1],  # point(y)
        camera_positions[i, 2],  # point(z)
        camera_directions[i, 0],  # vector(x)
        camera_directions[i, 1],  # vector(y)
        camera_directions[i, 2],  # vector(z)
        color='red',
        length=0.5,
        arrow_length_ratio=0.1,
        label='Camera Direction' if i == 0 else None
    )

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.legend()
plt.title('Camera Positions and Directions in 3D Space')
plt.show()

In [ ]:
testimg, testpose = images[10], poses[10]
plt.imshow(testimg)
plt.show()

In [ ]:
images = images[:100,...,:3] #images[:100, :, :, :3]
poses = poses[:100]

In [ ]:
print(testpose.shape)

# Compute camera position (world coordinates)
camera_to_world = np.linalg.inv(testpose)
cam_pos = camera_to_world[:3,3]
print(cam_pos)

# Define image plane corners in camera coordinates
f = 1.0  # focal length (distance along Z)
img_width = 1.0
img_height = 1.0

corners_cam = np.array([
    [-img_width/2, -img_height/2, f, 1],
    [ img_width/2, -img_height/2, f, 1],
    [ img_width/2,  img_height/2, f, 1],
    [-img_width/2,  img_height/2, f, 1],
]).T  # shape (4,4)

# Transform image plane corners to world coordinates
corners_world = camera_to_world @ corners_cam
corners_world = corners_world[:3, :].T  # shape (4,3)

# Plotting
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')

# Plot camera position
ax.scatter(*cam_pos, color='red', label='Camera Position')

# Plot image plane edges
for i in range(4):
    start = corners_world[i]
    end = corners_world[(i+1) % 4]
    ax.plot([start[0], end[0]], [start[1], end[1]], [start[2], end[2]], 'b-')

# Labels and title
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Camera position and image plane in 3D')

ax.legend()
plt.show()

---
---

## 🔧 Gaussian Splatting

In [ ]:
# Inverse sigmoid (logit) function
def inverse_sigmoid(x):
    return tf.math.log(x / (1.0 - x))

In [ ]:
class GaussianSplattingModel(tf.keras.Model):
    def __init__(self, num_gaussians=10000):
        super().__init__()
        self.num_gaussians = num_gaussians
        
        # Position initialization (3D coordinates)
        self.positions = tf.Variable(
            tf.random.uniform([num_gaussians, 3], -1.0, 1.0),
            name='positions'
        )
        
        # Rotation initialization (normalized quaternions)
        quats = tf.random.normal([num_gaussians, 4])
        self.rotations = tf.Variable(
            tf.keras.utils.normalize(quats, axis=-1),
            name='rotations'
        )
        
        # Scale initialization (log scale)
        self.scales = tf.Variable(
            tf.math.log(tf.random.uniform([num_gaussians, 3], 0.01, 0.1)),
            name='scales'
        )
        
        # Opacity initialization (inverse sigmoid)
        self.opacity = tf.Variable(
            self.inverse_sigmoid(tf.ones([num_gaussians, 1]) * 0.1),
            name='opacity'
        )
        
        # Color initialization (inverse sigmoid)
        self.colors = tf.Variable(
            self.inverse_sigmoid(tf.random.uniform([num_gaussians, 3])),
            name='colors'
        )
    
    def inverse_sigmoid(self, x):
        return tf.math.log(x / (1.0 - x + 1e-6) + 1e-6)
    
    def normalize_quaternions(self):
        self.rotations.assign(tf.keras.utils.normalize(self.rotations, axis=-1))
    
    def get_covariance_matrix(self):
        # Normalize quaternions first
        q = tf.keras.utils.normalize(self.rotations, axis=-1)
        w, x, y, z = q[:, 0], q[:, 1], q[:, 2], q[:, 3]
        
        # Build rotation matrix from quaternion
        R = tf.stack([
            tf.stack([1 - 2*(y**2 + z**2), 2*(x*y - w*z),     2*(x*z + w*y)], axis=1),
            tf.stack([2*(x*y + w*z),     1 - 2*(x**2 + z**2), 2*(y*z - w*x)], axis=1),
            tf.stack([2*(x*z - w*y),     2*(y*z + w*x),     1 - 2*(x**2 + y**2)], axis=1),
        ], axis=1)
        
        # Build scale matrix
        S = tf.linalg.diag(tf.exp(self.scales))
        
        # Compute covariance matrix
        RS = tf.linalg.matmul(R, S)
        cov = tf.linalg.matmul(RS, tf.transpose(RS, perm=[0, 2, 1]))
        return cov

    
#--------

    def render(self, screen_x, screen_y, depths, cov_2d, H, W):
        # Precompute pixel coordinates (cast to float32 to match center)
        y_coords, x_coords = tf.meshgrid(
            tf.range(tf.cast(H, tf.int32)), 
            tf.range(tf.cast(W, tf.int32)),
            indexing='ij'
        )
        pixel_coords = tf.cast(tf.stack([
            tf.reshape(x_coords, [-1]), 
            tf.reshape(y_coords, [-1])
        ], axis=1), tf.float32)  # <-- Cast to float32
        
        # Sort by depth
        depth_order = tf.argsort(depths, direction='DESCENDING')
        num_gaussians = tf.minimum(self.num_gaussians, 10000)
        
        # Initialize accumulators
        image = tf.zeros([H * W, 3], dtype=tf.float32)
        alpha_acc = tf.zeros([H * W], dtype=tf.float32)
        
        # Convert to while_loop compatible format
        def body(i, image, alpha_acc, continue_rendering):
            idx = depth_order[i]
            center = tf.stack([screen_x[idx], screen_y[idx]])  # Already float32
            
            # Boundary check
            in_bounds = tf.reduce_all([
                center[0] > -50.0,  # <-- Use float literals
                center[0] < tf.cast(W, tf.float32) + 50.0,
                center[1] > -50.0,
                center[1] < tf.cast(H, tf.float32) + 50.0
            ])
            
            def process_gaussian():
                diff_full = pixel_coords - center  # Now both float32
                dist2 = tf.reduce_sum(tf.square(diff_full), axis=1)
                mask = dist2 < (20.0 ** 2)
                active_coords = tf.boolean_mask(pixel_coords, mask)
                active_idx = tf.where(mask)[:, 0]
                
                cov = cov_2d[idx] + 1e-3 * tf.eye(2)
                inv_cov = tf.cond(
                    tf.abs(tf.linalg.det(cov)) > 1e-6,
                    lambda: tf.linalg.inv(cov),
                    lambda: tf.linalg.pinv(cov)
                )
                
                diff = active_coords - center
                maha_sq = tf.reduce_sum(
                    tf.multiply(tf.linalg.matmul(diff, inv_cov), diff), 
                    axis=1
                )
                weight = tf.exp(-0.5 * maha_sq)
                
                alpha = tf.sigmoid(self.opacity[idx, 0]) * weight
                transmittance = tf.gather(1.0 - alpha_acc, active_idx)
                alpha_blend = alpha * transmittance
                
                color = tf.sigmoid(self.colors[idx])
                delta_image = alpha_blend[:, None] * color[None, :]
                
                return (
                    tf.tensor_scatter_nd_add(image, active_idx[:, None], delta_image),
                    tf.tensor_scatter_nd_add(alpha_acc, active_idx[:, None], alpha_blend)
                )
            
            image_out, alpha_acc_out = tf.cond(
                in_bounds,
                process_gaussian,
                lambda: (image, alpha_acc)
            )
            
            # Early termination condition
            continue_rendering = tf.logical_and(
                continue_rendering,
                tf.reduce_mean(alpha_acc_out) <= 0.99
            )
            
            return i+1, image_out, alpha_acc_out, continue_rendering
        
        # Run the loop
        _, final_image, final_alpha, _ = tf.while_loop(
            cond=lambda i, img, acc, cont: tf.logical_and(i < num_gaussians, cont),
            body=body,
            loop_vars=(
                tf.constant(0), 
                image, 
                alpha_acc, 
                tf.constant(True)
            ),
            maximum_iterations=num_gaussians
        )
        
        return tf.reshape(final_image, [H, W, 3])

    def call(self, inputs, training=None):
        """
        Args:
            inputs: dict containing:
                - 'view_matrix': 4x4 view matrix (tf.float32)
                - 'focal': focal length (tf.float32)
                - 'height': image height (tf.int32)
                - 'width': image width (tf.int32)
        """
        # Ensure consistent data types for inputs
        view_matrix = tf.cast(inputs['view_matrix'], tf.float32)
        focal = tf.cast(inputs['focal'], tf.float32)
        H = tf.cast(inputs['height'], tf.float32)  # Convert to float32
        W = tf.cast(inputs['width'], tf.float32)   # Convert to float32
    
        # Perform rendering
        screen_x, screen_y, depths, cov_2d = self.project_gaussians(view_matrix, focal, H, W)
        return self.render(screen_x, screen_y, depths, cov_2d, H, W)
    
    def project_gaussians(self, view_matrix, focal, H, W):
        # Explicitly unify data types
        view_matrix = tf.cast(view_matrix, tf.float32)
        focal = tf.cast(focal, tf.float32)
        H = tf.cast(H, tf.float32)
        W = tf.cast(W, tf.float32)
    
        # Convert 3D positions to homogeneous coordinates
        positions_h = tf.concat([self.positions, tf.ones([self.num_gaussians, 1])], axis=1)
        cam_coords_h = tf.linalg.matmul(positions_h, tf.transpose(view_matrix))
        cam_coords = cam_coords_h[:, :3] / (cam_coords_h[:, 3:4] + 1e-8)  # Normalize by homogeneous w
        depths = cam_coords[:, 2]
    
        # Confirm and enforce float32 types
        cam_coords = tf.cast(cam_coords, tf.float32)
        depths = tf.cast(depths, tf.float32)
    
        # Project to screen coordinates
        screen_x = (cam_coords[:, 0] / (depths + 1e-8)) * focal + W / 2
        screen_y = (cam_coords[:, 1] / (depths + 1e-8)) * focal + H / 2
    
        # Compute 3D covariance matrices (user-defined method)
        cov_3d = self.get_covariance_matrix()
    
        # Transform covariance into camera space
        R_cam = view_matrix[:3, :3]
        cov_cam = tf.linalg.matmul(tf.linalg.matmul(R_cam, cov_3d), tf.transpose(R_cam))
    
        # Jacobian for perspective projection
        z_sq = depths ** 2
        J = tf.stack([
            tf.stack([focal / depths, tf.zeros_like(depths), -focal * cam_coords[:, 0] / z_sq], axis=-1),
            tf.stack([tf.zeros_like(depths), focal / depths, -focal * cam_coords[:, 1] / z_sq], axis=-1)
        ], axis=1)
    
        # Project covariance to 2D image space
        cov_2d = tf.linalg.matmul(tf.linalg.matmul(J, cov_cam), tf.transpose(J, perm=[0, 2, 1]))
        
        return screen_x, screen_y, depths, cov_2d


In [ ]:
@tf.function
def train_step(img_idx, pose, pixel_coords, target_rgb, model, focal, H, W, optimizer):
    target_rgb = tf.cast(target_rgb, tf.float32)

    with tf.GradientTape() as tape:
        inputs = {
            'view_matrix': tf.cast(tf.linalg.inv(pose), tf.float32),
            'focal': tf.cast(focal, tf.float32),
            'height': tf.cast(H, tf.float32), 
            'width': tf.cast(W, tf.float32)    
        }
        
        rendered_image = model(inputs)
        rendered_pixels = tf.gather_nd(rendered_image, pixel_coords)
        loss = tf.reduce_mean(tf.square(rendered_pixels - target_rgb))

    gradients = tape.gradient(loss, model.trainable_variables)
    gradients = [tf.clip_by_value(g, -1.0, 1.0) if g is not None else g for g in gradients]
    grads_and_vars = [(g, v) for g, v in zip(gradients, model.trainable_variables) if g is not None]

    if grads_and_vars:
        optimizer.apply_gradients(grads_and_vars)
    else:
        print("Warning: No gradients to apply!")

    model.normalize_quaternions()
    return loss, rendered_pixels

In [ ]:
# Gaussian Splatting Model old
# Training data preparation (reuse from NeRF)
def get_training_batch(images, poses, focal, H, W, N_rand=1024):
    """Randomly sample pixels for training"""
    # Randomly select an image
    img_i = np.random.choice(len(images))
    img = images[img_i]
    pose = poses[img_i]

    # Ensure pose is float32
    pose = tf.cast(pose, tf.float32)

    # Randomly sample pixels
    coords = tf.stack(tf.meshgrid(tf.range(H), tf.range(W), indexing='ij'), -1)
    coords = tf.reshape(coords, [-1, 2])
    select_inds = tf.random.uniform([N_rand], maxval=coords.shape[0], dtype=tf.int32)
    select_coords = tf.gather(coords, select_inds)

    target_batch = tf.gather_nd(img, select_coords)

    return img_i, pose, select_coords, target_batch

In [ ]:
# Training parameters
N_iters = 1000
N_rand = 1024
learning_rate = 0.008 #1e-3,1e-4
testimg_idx = 10

# Initialize model
model = GaussianSplattingModel(num_gaussians=1000)

# Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate)

# Training loop
print("Starting Gaussian Splatting training...")
print(f"Number of Gaussians: {model.num_gaussians}")
print(f"Number of training iterations: {N_iters}")

In [ ]:
psnr_list = []
loss_list = []
best_psnr = 0
best_weights = None

for i in tqdm(range(N_iters), desc="Training"):
    # Training batch
    img_i, pose, pixel_coords, target_rgb = get_training_batch(images, poses, focal, H, W, N_rand)
    
    # Training step
    loss, rendered_pixels = train_step(img_i, pose, pixel_coords, target_rgb, model, focal, H, W, optimizer)
    loss_list.append(loss.numpy())

    # Evaluation
    if i % 100 == 0:
        tqdm.write(f"Step {i:06d}: Loss = {loss:.4f}")
        
        if i % 500 == 0:
            # Prepare test inputs
            test_pose = poses[testimg_idx]
            inputs = {
                'view_matrix': tf.cast(tf.linalg.inv(test_pose), tf.float32),
                'focal': tf.cast(focal, tf.float32),
                'height': tf.cast(H, tf.float32),
                'width': tf.cast(W, tf.float32)
            }
            
            # Render using the model directly
            rendered_test = model(inputs)
            target_test = images[testimg_idx]
            
            # Calculate PSNR
            mse = tf.reduce_mean(tf.square(rendered_test - target_test))
            psnr = -10. * tf.math.log(mse) / tf.math.log(10.)
            psnr_list.append(psnr.numpy())
            
            # Visualization
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
            ax1.imshow(target_test)
            ax1.set_title("Ground Truth")
            ax2.imshow(np.clip(rendered_test, 0, 1))
            ax2.set_title(f"Iter {i}, PSNR: {psnr:.2f}")
            plt.show()
            plt.close(fig)


In [ ]:
# Plot training curves
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(loss_list)
plt.title("Training Loss")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.yscale('log')

plt.subplot(1, 2, 2)
plt.plot(range(0, len(psnr_list) * 500, 500), psnr_list)
plt.title("Test PSNR")
plt.xlabel("Iteration")
plt.ylabel("PSNR (dB)")
plt.tight_layout()
plt.show()

# Final test rendering
print("Performing final test rendering...")

final_rendered = model({
    'view_matrix': tf.constant(poses[testimg_idx], dtype=tf.float32),
    'focal': tf.constant(focal, dtype=tf.float32),
    'height': tf.constant(H, dtype=tf.float32),
    'width': tf.constant(W, dtype=tf.float32)
})

final_rendered = tf.cast(final_rendered, tf.float32).numpy()
target_image = tf.cast(images[testimg_idx], tf.float32).numpy() if isinstance(images[testimg_idx], tf.Tensor) else images[testimg_idx].astype(np.float32)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(images[testimg_idx])
plt.title("Ground Truth")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(np.clip(final_rendered, 0, 1))
plt.title("Gaussian Splatting Rendered")
plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# Final PSNR calculation
final_mse = tf.reduce_mean(tf.square(final_rendered - target_image))
final_psnr = -10. * tf.math.log(final_mse) / tf.math.log(tf.constant(10., dtype=tf.float32))
print(f"Final PSNR: {final_psnr:.2f} dB")

---
---

## 🔧 Interactive Visualization

In [ ]:
print(H,W)

In [ ]:
# Camera pose generation functions (used as-is)
trans_t = lambda t : tf.convert_to_tensor([
    [1,0,0,0],
    [0,1,0,0],
    [0,0,1,t],
    [0,0,0,1],
], dtype=tf.float32)

rot_phi = lambda phi : tf.convert_to_tensor([
    [1,0,0,0],
    [0,tf.cos(phi),-tf.sin(phi),0],
    [0,tf.sin(phi), tf.cos(phi),0],
    [0,0,0,1],
], dtype=tf.float32)

rot_theta = lambda th : tf.convert_to_tensor([
    [tf.cos(th),0,-tf.sin(th),0],
    [0,1,0,0],
    [tf.sin(th),0, tf.cos(th),0],
    [0,0,0,1],
], dtype=tf.float32)

def pose_spherical(theta, phi, radius):
    c2w = trans_t(radius)
    c2w = rot_phi(phi / 180. * np.pi) @ c2w
    c2w = rot_theta(theta / 180. * np.pi) @ c2w
    c2w = np.array([[-1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]]) @ c2w
    return c2w


# Rendering function for Gaussian Splatting
def render_view(model, c2w, H, W, focal):
    inputs = {
        'view_matrix': tf.cast(tf.linalg.inv(c2w), tf.float32),
        'focal': tf.cast(focal, tf.float32),
        'height': tf.cast(H//5, tf.float32),  # float32に統一
        'width': tf.cast(W//5, tf.float32)    # float32に統一
    }
    return model(inputs).numpy()

def f(**kwargs):
    c2w = pose_spherical(**kwargs)
    # Direct rendering instead of get_rays
    img = render_view(model, c2w, H, W, focal)

    plt.figure(2, figsize=(20, 6))
    plt.imshow(np.clip(img, 0, 1))
    plt.show()

# Slider configuration
sldr = lambda v, mi, ma: widgets.FloatSlider(
    value=v,
    min=mi,
    max=ma,
    step=.01,
)

names = [
    ['theta', [100., 0., 360]],
    ['phi', [-30., -90, 0]],
    ['radius', [4., 3., 5.]],
]

interactive_plot = interactive(f, **{s[0]: sldr(*s[1]) for s in names})
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot


## 🔧 Render 360 Video

In [ ]:
from tqdm.notebook import tqdm

frames = []
for th in tqdm(np.linspace(0., 360., 36, endpoint=False)):
    c2w = pose_spherical(th, -30., 4.)
    img = render_view(model, c2w, H//5, W//5, focal)
    frames.append((255*np.clip(img, 0, 1)).astype(np.uint8))

import imageio
f = 'video.mp4'
imageio.mimwrite(f, frames, fps=4, quality=10)

from IPython.display import HTML
from base64 import b64encode
mp4 = open('video.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls autoplay loop>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)